In [1]:
import pylab as plt
import pymoc
import xidplus
import numpy as np
%matplotlib inline
from astropy.table import Table, join

In [2]:
import seaborn as sns

This notebook uses all the raw data from the CIGALE predictions and photoz catalogue, maps, PSF and relevant MOCs to create XID+ prior object and relevant tiling scheme

## Read in MOCs
The selection functions required are the main MOC associated with the masterlist. 

In [3]:
Sel_func=pymoc.MOC()
Sel_func.read('../../dmu4/dmu4_sm_SGP/data/holes_SGP_vista_ks_20180509_O16_MOC.fits')

## Read in CIGALE predictions catalogue

In [4]:
cigale=Table.read('../../dmu28/dmu28_SSDF/data/hatlas_ssdf_Ldust_prediction_results.fits')

In [5]:
cigale['id'].name = 'help_id'

## Read in photoz

In [6]:
photoz=Table.read('../../dmu24/dmu24_SSDF/data/master_catalogue_ssdf_20180221_photoz_20180612_r_optimised.fits')

In [7]:
photoz

help_id,RA,DEC,id,z1_median,z1_min,z1_max,z1_area,z2_median,z2_min,z2_max,z2_area,za_hb,chi_r_eazy,chi_r_atlas,chi_r_cosmos,chi_r_stellar,stellar_type
str27,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str6
HELP_J000034.399-292559.520,0.143327990467,-29.4332000185,749,0.1817,0.066,0.3068,0.795,-99.0,-99.0,-99.0,-99.0,0.159257182798,0.23267745,0.10496575,0.72783425,8.409065,k4i
HELP_J000023.158-295010.568,0.0964932450257,-29.8362688223,1027,0.4319,0.1697,0.7112,0.795,-99.0,-99.0,-99.0,-99.0,0.404234079574,-99.0,-99.0,-99.0,-99.0,
HELP_J000005.462-295122.335,0.0227600836353,-29.8562042667,1085,0.5612,0.2162,0.929,0.797,-99.0,-99.0,-99.0,-99.0,0.522525938906,-99.0,-99.0,-99.0,-99.0,
HELP_J000051.301-291551.140,0.213755113372,-29.2642055793,1395,0.6162,0.2345,1.0237,0.799,-99.0,-99.0,-99.0,-99.0,0.568823299054,-99.0,-99.0,-99.0,-99.0,
HELP_J000007.372-304303.633,0.0307178887805,-30.7176758651,1426,0.7514,0.5877,0.9061,0.79,-99.0,-99.0,-99.0,-99.0,0.789843601218,12.7088585714,10.3248642857,9.94852428571,4.85362714286,m4v
HELP_J000007.404-304200.072,0.030848028776,-30.7000199487,1429,0.4737,0.2198,0.7112,0.795,-99.0,-99.0,-99.0,-99.0,0.438291695655,0.260860428571,0.434253142857,0.400822857143,0.740359,k7v
HELP_J000007.125-305610.210,0.029689012927,-30.9361693546,1430,0.8164,0.3068,1.3648,0.797,-99.0,-99.0,-99.0,-99.0,0.721483855642,0.70108575,1.13647675,0.69995475,2.820445,m0iii
HELP_J000007.269-304335.704,0.0302890794322,-30.7265845696,1437,0.9796,0.3587,1.642,0.797,-99.0,-99.0,-99.0,-99.0,0.94644083855,0.518284,0.47864975,0.2708165,3.6958875,m6iii
HELP_J000007.195-304537.048,0.0299783907374,-30.7602909921,1438,0.3343,0.1662,0.4999,0.796,-99.0,-99.0,-99.0,-99.0,0.42116086894,0.65577025,1.40075625,2.11645,5.7564925,wk0iii


## Join CIGALE and photoz tables

In [8]:
prior=join(cigale,photoz)

In [9]:
from astropy.cosmology import Planck15 as cosmo
from astropy import units as u
f_pred=prior['bayes.dust.luminosity']/(4*np.pi*cosmo.luminosity_distance(prior['z1_median']).to(u.cm))

In [10]:
prior=prior[np.isfinite(f_pred.value)][np.log10(f_pred.value[np.isfinite(f_pred.value)])>8.5]

In [11]:
prior['DEC'].name='Dec'

In [13]:
len(cigale)

4436690

## Read in Maps

In [14]:
pswfits='../../dmu19/dmu19_HELP-SPIRE-maps/data/SSDF_SPIRE250_v1.0.fits'#SPIRE 250 map
pmwfits='../../dmu19/dmu19_HELP-SPIRE-maps/data/SSDF_SPIRE350_v1.0.fits'#SPIRE 350 map
plwfits='../../dmu19/dmu19_HELP-SPIRE-maps/data/SSDF_SPIRE500_v1.0.fits'#SPIRE 500 map

#output folder
output_folder='./'

In [15]:
from astropy.io import fits
from astropy import wcs

#-----250-------------
hdulist = fits.open(pswfits)
im250phdu=hdulist[0].header
im250hdu=hdulist[1].header

im250=hdulist[1].data*1.0E3 #convert to mJy
nim250=hdulist[3].data*1.0E3 #convert to mJy
w_250 = wcs.WCS(hdulist[1].header)
pixsize250=3600.0*w_250.wcs.cdelt #pixel size (in arcseconds)
hdulist.close()
#-----350-------------
hdulist = fits.open(pmwfits)
im350phdu=hdulist[0].header
im350hdu=hdulist[1].header

im350=hdulist[1].data*1.0E3 #convert to mJy
nim350=hdulist[3].data*1.0E3 #convert to mJy
w_350 = wcs.WCS(hdulist[1].header)
pixsize350=3600.0*w_350.wcs.cdelt #pixel size (in arcseconds)
hdulist.close()
#-----500-------------
hdulist = fits.open(plwfits)
im500phdu=hdulist[0].header
im500hdu=hdulist[1].header
im500=hdulist[1].data*1.0E3 #convert to mJy
nim500=hdulist[3].data*1.0E3 #convert to mJy
w_500 = wcs.WCS(hdulist[1].header)
pixsize500=3600.0*w_500.wcs.cdelt #pixel size (in arcseconds)
hdulist.close()

In [16]:
## Set XID+ prior class

In [17]:
#---prior250--------
prior250=xidplus.prior(im250,nim250,im250phdu,im250hdu, moc=Sel_func)#Initialise with map, uncertianty map, wcs info and primary header
prior250.prior_cat(prior['RA'] ,prior['Dec'] ,'hatlas_sgp_Ldust_prediction_results.fits',ID=prior['help_id'] )#Set input catalogue
prior250.prior_bkg(-5.0,5)#Set prior on background (assumes Gaussian pdf with mu and sigma)
#---prior350--------
prior350=xidplus.prior(im350,nim350,im350phdu,im350hdu, moc=Sel_func)
prior350.prior_cat(prior['RA'] ,prior['Dec'] ,'hatlas_sgp_Ldust_prediction_results.fits',ID=prior['help_id'] )
prior350.prior_bkg(-5.0,5)

#---prior500--------
prior500=xidplus.prior(im500,nim500,im500phdu,im500hdu, moc=Sel_func)
prior500.prior_cat(prior['RA'] ,prior['Dec'] ,'hatlas_sgp_Ldust_prediction_results.fits',ID=prior['help_id'] )
prior500.prior_bkg(-5.0,5)

In [18]:
#pixsize array (size of pixels in arcseconds)
pixsize=np.array([pixsize250,pixsize350,pixsize500])
#point response function for the three bands
prfsize=np.array([18.15,25.15,36.3])
#use Gaussian2DKernel to create prf (requires stddev rather than fwhm hence pfwhm/2.355)
from astropy.convolution import Gaussian2DKernel

##---------fit using Gaussian beam-----------------------
prf250=Gaussian2DKernel(prfsize[0]/2.355,x_size=101,y_size=101)
prf250.normalize(mode='peak')
prf350=Gaussian2DKernel(prfsize[1]/2.355,x_size=101,y_size=101)
prf350.normalize(mode='peak')
prf500=Gaussian2DKernel(prfsize[2]/2.355,x_size=101,y_size=101)
prf500.normalize(mode='peak')

pind250=np.arange(0,101,1)*1.0/pixsize[0,1] #get 250 scale in terms of pixel scale of map
pind350=np.arange(0,101,1)*1.0/pixsize[1,1] #get 350 scale in terms of pixel scale of map
pind500=np.arange(0,101,1)*1.0/pixsize[2,1] #get 500 scale in terms of pixel scale of map

prior250.set_prf(prf250.array,pind250,pind250)#requires psf as 2d grid, and x and y bins for grid (in pixel scale)
prior350.set_prf(prf350.array,pind350,pind350)
prior500.set_prf(prf500.array,pind500,pind500)

In [19]:
import pickle
#from moc, get healpix pixels at a given order
from xidplus import moc_routines
order=9
tiles=moc_routines.get_HEALPix_pixels(order,prior250.sra,prior250.sdec,unique=True)
order_large=6
tiles_large=moc_routines.get_HEALPix_pixels(order_large,prior250.sra,prior250.sdec,unique=True)
print('----- There are '+str(len(tiles))+' tiles required for input catalogue and '+str(len(tiles_large))+' large tiles')
output_folder='./'
outfile=output_folder+'Master_prior.pkl'
xidplus.io.pickle_dump({'priors':[prior250,prior350,prior500],'tiles':tiles,'order':order,'version':xidplus.io.git_version()},outfile)
outfile=output_folder+'Tiles.pkl'
with open(outfile, 'wb') as f:
    pickle.dump({'tiles':tiles,'order':order,'tiles_large':tiles_large,'order_large':order_large,'version':xidplus.io.git_version()},f)
raise SystemExit()

----- There are 18279 tiles required for input catalogue and 379 large tiles
writing total_bytes=5118646130...
writing bytes [0, 1073741824)... done.
writing bytes [1073741824, 2147483648)... done.
writing bytes [2147483648, 3221225472)... done.
writing bytes [3221225472, 4294967296)... done.
writing bytes [4294967296, 5118646130)... done.


SystemExit: 

In [57]:
prior250.nsrc

1236678